<a href="https://colab.research.google.com/github/mikmikthemimic/PyTorch-Object-Detection-Faster-RCNN-Tutorial/blob/master/Get_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/mikmikthemimic/PyTorch-Object-Detection-Faster-RCNN-Tutorial.git

Cloning into 'PyTorch-Object-Detection-Faster-RCNN-Tutorial'...
remote: Enumerating objects: 928, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 928 (delta 314), reused 417 (delta 288), pack-reused 457
Receiving objects: 100% (928/928), 191.98 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (511/511), done.


In [ ]:
%%capture
import os

if not os.path.isdir('/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/data/heads/input/'):
  !unzip '/content/drive/MyDrive/raw dataset/fin_dataset.zip' -d '/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/data/heads'

In [ ]:
%cd /content/PyTorch-Object-Detection-Faster-RCNN-Tutorial

/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial


In [ ]:
!pip install -q -r requirements_colab.txt

In [ ]:
#new change   08/06/2024

# Wag na muna eto
# !pip install pydantic==1.8.1
# !pip install pydantic-settings

  Using cached pydantic-1.8.1-py3-none-any.whl.metadata (100 kB)
Using cached pydantic-1.8.1-py3-none-any.whl (125 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.12 requires pydantic>=2.7.0, but you have pydantic 1.8.1 which is incompatible.
confection 0.1.5 requires pydantic!=1.8,!=1.8.1,<3.0.0,>=1.7.4, but you have pydantic 1.8.1 which is incompatible.
napari 0.5.1 requires pydantic>=1.9.0, but you have pydantic 1.8.1 which is incompatible.
pydantic-settings 2.4.0 requires pydantic>=2.7.0, but you have pydantic 1.8.1 which is incompatible.
spacy 3.7.5 requires pydantic!=1.8,!=1.8.1,<3.0.0,>=1.7.4, but you have pydantic 1.8.1 which is incompatible.
thinc 8.2.5 requires pydantic!=

  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.1
    Uninstalling pydantic-1.8.1:
      Successfully uninstalled pydantic-1.8.1


In [ ]:
# Wag na muna eto
# !pip install "napari[all]"

In [ ]:
!git pull

Already up to date.


In [ ]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

#new change   08/06/2024      attempt to fix error
#from pydantic_settings import BaseSettings  # Import from pydantic_settings

from pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones
from pytorch_faster_rcnn_tutorial.datasets import (
    ObjectDetectionDataSet,
    ObjectDetectionDatasetSingle,
)
from pytorch_faster_rcnn_tutorial.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn_tutorial.transformations import (
    ComposeDouble,
    ComposeSingle,
    FunctionWrapperDouble,
    FunctionWrapperSingle,
    apply_nms,
    apply_score_threshold,
    normalize_01,
)
from pytorch_faster_rcnn_tutorial.utils import (
    collate_single,
    get_filenames_of_path,
    save_json,
)
# from pytorch_faster_rcnn_tutorial.viewers.object_detection_viewer import (
#     ObjectDetectionViewer,
#     ObjectDetectionViewerSingle,
# )
# from training_script import NeptuneSettings    #new change   08/06/2024

# excluding training_script bc yung neptune settings is being called sa code block here

In [ ]:
# variables updated

params = {
    "EXPERIMENT": "GMT3-488",  # experiment name, e.g. Head-42
    "OWNER": "mikmikthemimic",  # e.g. johndoe55
    "INPUT_DIR": "pytorch_faster_rcnn_tutorial/data/heads/test",  # files to predict
    "PREDICTIONS_PATH": "predictions",  # where to save the predictions
    "MODEL_DIR": "/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/model/kfolds/fold_4.ckpt",  # load model from checkpoint
    "DOWNLOAD": False,  # whether to download from neptune
    "DOWNLOAD_PATH": "pytorch_faster_rcnn_tutorial/model/kfolds",  # where to save the model if DOWNLOAD is True
    "PROJECT": "GM-Thesis3",  # Project name

    "ENSEMBLE": False,  # nilagay ko lang to for the ensemble model
}

In [ ]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params["INPUT_DIR"]))
inputs.sort()
print(f"Found {len(inputs)} files in {params['INPUT_DIR']}")

Found 640 files in pytorch_faster_rcnn_tutorial/data/heads/test


In [ ]:
# transformations
transforms = ComposeSingle(
    [
        FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
        FunctionWrapperSingle(normalize_01),
    ]
)

In [ ]:
# create dataset
dataset = ObjectDetectionDatasetSingle(
    inputs=inputs,
    transform=transforms,
    use_cache=False,
)


In [ ]:
# create dataloader
dataloader_prediction = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_single,
)


In [ ]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
print(f"Project: {project_name}")
project = neptune.init_run(
    project=project_name,
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNmE3MzZjYS01MDUxLTQ2YjYtOGU3ZC04YzQwNjg4NmJiZDcifQ==",
    with_id=params["EXPERIMENT"],
    mode="read-only",
)  # get project
experiment_id = params["EXPERIMENT"]  # experiment id
parameters = project['training/hyperparams'].fetch()


Project: mikmikthemimic/GM-Thesis3
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mikmikthemimic/GM-Thesis3/e/GMT3-488


In [ ]:
print(project['training/hyperparams'].fetch())

{'ACCELERATOR': 'cuda', 'ANCHOR_SIZE': '((32,), (64,), (128,), (256,))', 'ASPECT_RATIOS': '((0.5, 1.0, 2.0),)', 'BACKBONE': 'ResNetBackbones.RESNET34', 'BATCH_SIZE': 20, 'CACHE': False, 'CLASSES': 6, 'FAST_DEV_RUN': False, 'FOLDS': 5, 'FPN': True, 'IMG_MEAN': '[0.485, 0.456, 0.406]', 'IMG_STD': '[0.229, 0.224, 0.225]', 'IOU_THRESHOLD': 0.6, 'LOG_MODEL': True, 'LR': 0.0025, 'MAXEPOCHS': 12, 'MAX_SIZE': 1025, 'MIN_SIZE': 1024, 'PATIENCE': 50, 'PRECISION': 32, 'SAVE_DIR': '/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial', 'SEED': 42, 'iou_threshold': 0.6, 'lr': 0.0025, 'model': 'None'}


In [ ]:
# rcnn transform
transform = GeneralizedRCNNTransform(
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
    image_mean=ast.literal_eval(parameters["IMG_MEAN"]),
    image_std=ast.literal_eval(parameters["IMG_STD"]),
    box_nms_thresh = 0.6
)

In [ ]:
# color mapping
color_mapping = {
    1: "blue",
    2: "green",
    3: "white",
    4: "yellow",
    5: "red"
}

In [ ]:
if not os.path.isdir('/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/model/kfolds'):
  !mkdir -p '/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/model/kfolds'
!cp '/content/drive/MyDrive/08-06-2024/fold_4.ckpt' '/content/PyTorch-Object-Detection-Faster-RCNN-Tutorial/pytorch_faster_rcnn_tutorial/model/kfolds'

In [ ]:
# download model from neptune or load from checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if params["DOWNLOAD"]:
    download_path = pathlib.Path(os.getcwd()) / params["DOWNLOAD_PATH"]
    download_path.mkdir(parents=True, exist_ok=True)
    if params["ENSEMBLE"]:
        model_name = "ensemble_model.pt"  # that's how I called the best model
        # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
        if not (download_path / model_name).is_file():
            project['artifacts/ensemble_model'].download(
                destination=download_path.as_posix()
            )  # download model

        model_state_dict = torch.load(
            download_path / model_name, map_location=device
        )
    else:
        #new change   08/06/2024      updated to load checkpoint
        model_name = "fold_4.ckpt"  # that's how I called the best model                      best-model.pt
        # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
        if not (download_path / model_name).is_file():
            project['artifacts/fold_4.ckpt'].download(
                destination=download_path.as_posix()
            )  # download model

        model_state_dict = torch.load(
            download_path / model_name, map_location=device
        )
else:
    checkpoint = torch.load(params["MODEL_DIR"], map_location=device)
    print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])


In [ ]:
model = get_faster_rcnn_resnet(
    num_classes=int(parameters["CLASSES"]),
    backbone_name=ResNetBackbones(parameters["BACKBONE"].split(".")[1].lower()),  # reverse look-up enum
    anchor_size=ast.literal_eval(parameters["ANCHOR_SIZE"]),
    aspect_ratios=ast.literal_eval(parameters["ASPECT_RATIOS"]),
    fpn=ast.literal_eval(str(parameters['FPN'])),
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
if 'state_dict' in checkpoint.keys():
  checkpoint = checkpoint['state_dict']

In [ ]:
# Check if keys start with model. and remove it
model_state_dict = {k.replace("model.", ""): v for k, v in checkpoint.items() if k.startswith("model.")}
print(model_state_dict.keys())

dict_keys(['backbone.body.conv1.weight', 'backbone.body.bn1.weight', 'backbone.body.bn1.bias', 'backbone.body.bn1.running_mean', 'backbone.body.bn1.running_var', 'backbone.body.layer1.0.conv1.weight', 'backbone.body.layer1.0.bn1.weight', 'backbone.body.layer1.0.bn1.bias', 'backbone.body.layer1.0.bn1.running_mean', 'backbone.body.layer1.0.bn1.running_var', 'backbone.body.layer1.0.conv2.weight', 'backbone.body.layer1.0.bn2.weight', 'backbone.body.layer1.0.bn2.bias', 'backbone.body.layer1.0.bn2.running_mean', 'backbone.body.layer1.0.bn2.running_var', 'backbone.body.layer1.1.conv1.weight', 'backbone.body.layer1.1.bn1.weight', 'backbone.body.layer1.1.bn1.bias', 'backbone.body.layer1.1.bn1.running_mean', 'backbone.body.layer1.1.bn1.running_var', 'backbone.body.layer1.1.conv2.weight', 'backbone.body.layer1.1.bn2.weight', 'backbone.body.layer1.1.bn2.bias', 'backbone.body.layer1.1.bn2.running_mean', 'backbone.body.layer1.1.bn2.running_var', 'backbone.body.layer1.2.conv1.weight', 'backbone.body.

In [ ]:
# load weights
print("Printing Model State Dict Keys:")
for key in model_state_dict.keys():
    print(key)

model.load_state_dict(model_state_dict)

Printing Model State Dict Keys:
backbone.body.conv1.weight
backbone.body.bn1.weight
backbone.body.bn1.bias
backbone.body.bn1.running_mean
backbone.body.bn1.running_var
backbone.body.layer1.0.conv1.weight
backbone.body.layer1.0.bn1.weight
backbone.body.layer1.0.bn1.bias
backbone.body.layer1.0.bn1.running_mean
backbone.body.layer1.0.bn1.running_var
backbone.body.layer1.0.conv2.weight
backbone.body.layer1.0.bn2.weight
backbone.body.layer1.0.bn2.bias
backbone.body.layer1.0.bn2.running_mean
backbone.body.layer1.0.bn2.running_var
backbone.body.layer1.1.conv1.weight
backbone.body.layer1.1.bn1.weight
backbone.body.layer1.1.bn1.bias
backbone.body.layer1.1.bn1.running_mean
backbone.body.layer1.1.bn1.running_var
backbone.body.layer1.1.conv2.weight
backbone.body.layer1.1.bn2.weight
backbone.body.layer1.1.bn2.bias
backbone.body.layer1.1.bn2.running_mean
backbone.body.layer1.1.bn2.running_var
backbone.body.layer1.2.conv1.weight
backbone.body.layer1.2.bn1.weight
backbone.body.layer1.2.bn1.bias
backbo

<All keys matched successfully>

In [ ]:
# inference (cpu)
model.eval()
model.to(device)
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        # Move tensors to CPU before converting to NumPy
        pred = {key: value.cpu().numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {
            key: value.tolist() for key, value in pred.items()
        }  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix(".json"))

In [ ]:
from datetime import datetime

# Set the filename as predictions_time
filename = f'predictions_{params["EXPERIMENT"]}_{datetime.today().strftime("%Y_%m_%d")}.zip'

!zip -r {filename} predictions
!cp {filename} "/content/drive/MyDrive"

NameError: name 'params' is not defined

In [ ]:
from datetime import datetime
filename = f'model_statedict_{params["EXPERIMENT"]}_{datetime.today().strftime("%Y_%m_%d")}.pt'

torch.save(model.state_dict(), filename)

!cp {filename} /content/drive/MyDrive/{filename}